### 链表

在上一章我们谈谈到了array,array有如下缺陷:
1. 动态数组的长度可能比实际所需的长度大
2. 因为需要resize,在实际应用中cost很大
3. 在前面插入和删除的成本太高  

链表(linked list)可以作为array的代替,依赖于一个节点(node)对象,每个nodo包含两个应用,指向它自身的元素,以及指向下一个node,链表index的cost很高,但避免了array的三个缺陷.

#### 单链链表
A singly linked list, in its simplest form, is a collection of nodes that collectively form a linear sequence. Each node stores a reference to an object that is an element of the sequence, as well as a reference to the next node of the list.
![Screenshot from 2017-03-24 10-59-48.png](https://ooo.0o0.ooo/2017/03/24/58d48bb65233c.png)
##### 单链链表就是线性的一个node连着一个node
![Screenshot from 2017-03-24 11-01-42.png](https://ooo.0o0.ooo/2017/03/24/58d48c2708fca.png)
尾元素指向一个空集,根据node的指针指向下一个node,称为链表遍历,指针跳跃(link hopping)

##### 链表必须保存对头部的引用,没有头部无法遍历链表
##### 链表的插入操作
![Screenshot from 2017-03-24 11-10-04.png](https://ooo.0o0.ooo/2017/03/24/58d48e2160c60.png)
```
#从头部插入的代码
add_first(L,e):
    newest=Node(e)
    newest.next=L.head
    L.head=newest
    L.size=L.size+1
```

##### 单链链表的元素移除
![Screenshot from 2017-03-24 11-19-47.png](https://ooo.0o0.ooo/2017/03/24/58d49065d6193.png)
```
remove first(L):
    if L.head is None:
        raise error
    L.head=L.head.next
    L.size-=1
```

##### 用链表实现一个栈

In [13]:
class LinkedStack:
    
    class _Node:
        __slots__="_item","_next"
        def __init__(self,item,next):
            self._item=item
            self._next=next
            
    def __init__(self):
        self._head=None
        self._size=0
        
    def push(self,item):
        newnode=self._Node(item,self._head)
        self._head=newnode
        self._size+=1
        
    def pop(self):
        if self.isempty():
            raise Empty("stack is empty")
        self._head=self._head._next
        self._size-=1
        
    def isempty(self):
        return self._siez==0
    
    def __len__(self):
        return self._size
    
    #每次调用iter,新建一个self.curhead引用当前head,否则链表不能复用
    def __iter__(self):
        self.curhead=self._head
        return self

    def __next__(self):
        if self.curhead is None:
            raise  StopIteration
        else:
            item=self.curhead._item
            self.curhead=self.curhead._next
            return item
        
    def __str__(self):
        return str([i for i in iter(self)])
            
           
l=LinkedStack()
l.push(1)
l.push(2)

print(l)
l.pop()
print(l)

[2, 1]
[1]


链表实现的栈的成本分析
![Screenshot from 2017-03-24 13-28-39.png](https://ooo.0o0.ooo/2017/03/24/58d4ae9ee5c94.png)

链表实现一个队列,需要一个head指向头部,一个tail指向尾部
1. 第一次插入时,head指向插入的node 如self.head=firstnode;tail也指向插入的node,如self.tail=firstnode
2. 之后的每次插入,只修改tail,将tail.next指向新节点(注意tail一直是以head的node为首的队列的最后一个node,tail.next指向新节点,队列自然增长),然后将tail指向新插入的node.

In [21]:
class LinkQueue:
    class Node:
        __slots__="item","next"
        def __init__(self,item,next):
            self.item=item
            self.next=next
            
    def __init__(self):
        self.size=0
        self.head=None
        self.tail=None
    
    def isempty(self):
        return self.size==0
    
    def enqueue(self,item):
        nownode=self.Node(item,None)
        if self.isempty():
            self.head=nownode
        else:
            #尾节点连接新节点
            self.tail.next=nownode
        #尾指针指向新节点
        self.tail=nownode
        self.size+=1
        
    def dequeue(self):
        if self.isempty():
            raise Empty("no item")
        item=self.head.item
        self.head=self.head.next
        self.size-=1
        #如此时队列已空,那么尾节点就不存在了
        if self.isempty():
            self.tail=None
        return item
    
    
    def __len__(self):
        return self.size
    
    def __iter__(self):
        self.curhead=self.head
        return self

    def __next__(self):
        if self.curhead is None:
            raise  StopIteration
        else:
            item=self.curhead.item
            self.curhead=self.curhead.next
            return item
        
    def __str__(self):
        return str([i for i in iter(self)])
    
l=LinkQueue()
l.enqueue(1)
print(l)
l.enqueue(2)
print(l)
l.dequeue()
print(l)
l.dequeue()
print(l)

[1]
[1, 2]
[2]
[]


#### 循环链表
![Screenshot from 2017-03-24 14-30-29.png](https://ooo.0o0.ooo/2017/03/24/58d4bd1579fb3.png)
循环链表定义一个current去形容节点,通过设置current=current.next进行遍历.
循环列表的一个应用是轮转调度算法(Round-Robin Schedulers)  
普通队列需要
```
1. e=Q.dequeue()
2. Service e
3. Q.enqueue() 
``` 
而循环队列需要
```
1. Service element Q.front()
2. Q.rotate()
```
步骤更少,而且无需新建节点

实现只使用tail和size,用tail.next指向头,入列时,新元素.next指向tail.next,tail.next指向新元素,新元素变为tail

In [23]:
class CircularQueue:
    
    class Node:
        def __init__(self,item,next):
            self.item=item
            self.next=next
        
    def __init__(self):
        self.tail=None
        self.size=0
        
    def isempty(self):
        return self.size==0
    
    def enqueue(self,item):
        newnode=self.Node(item,None)
        if self.isempty():
            #只有一个元素,头就是尾
            newnode.next=newnode
        else:
            #self.tail.next是head,newnode指向head
            newnode.next=self.tail.next
            #oldnode的next指向新元素
            self.tail.next=newnode
        #tail永远是新元素
        self.tail=newnode
        self.size+=1
            
    def dequeue(self):
        if self.isempty():
            raise Empty("no item")
        oldhead=self.tail.next
        item=oldhead.item
        #只剩一个元素,归零
        if self.size==1:
            self.tail=None
        else:
            self.tail.next=oldhead.next
        self.size-=1
        return item
    
    def __len__(self):
        return self.size
    
    #旋转,首元素变成尾元素
    def rotate(self):
        if self.size>0:
            self.tail=self.tail.next

#### 双向链表
不仅包含一个向后的指针,还包含一个向前的指针:
双链链表的header 和 trailer是特殊的,它们是哨兵,不存储任何元素,只是用来确定边界
![Screenshot from 2017-03-24 15-38-40.png](https://ooo.0o0.ooo/2017/03/24/58d4cd1584459.png)

使用哨兵的优势:
1. 极大简化操作逻辑,头和尾始终不动,只有中间的节点变化
2. 所有插入和删除都能同样的实现,无需考虑空列的情况
插入节点
![Screenshot from 2017-03-24 15-48-41.png](https://ooo.0o0.ooo/2017/03/24/58d4cf755f83f.png)
删除节点
![Screenshot from 2017-03-24 15-49-30.png](https://ooo.0o0.ooo/2017/03/24/58d4cf9d0ff76.png)

In [32]:
class DoubleLinkedBase:
    class Node:
        def __init__(self,item,pre,next):
            self.item=item
            self.pre=pre
            self.next=next
    def __init__(self):
        self.head=self.Node(None,None,None)
        self.tail=self.Node(None,None,None)
        self.head.next=self.tail
        self.tail.pre=self.head
        self.size=0
        
    def isempty(self):
        return self.size==0
    
    def insert_between(self,item,pre,next):
        nownode=self.Node(item,pre,next)
        pre.next=nownode
        next.pre=nownode
        self.size+=1
        return nownode
    
    def delete_node(self,node):
        pre=node.pre
        next=node.next
        pre.next=next
        next.pre=pre
        self.size-=1
        item=node.item
        node.pre=node.next=node.item=None
        return item
########################################   
    #增加从前插入,删除的方法就成了一个栈
    def insert_first(self,item):
        return self.insert_between(item,self.head,self.head.next)
    
    
    def delete_first(self):
        if self.isempty():
            raise empty("no item")
        else:
            return self.delete_node(self.head.next)
        
    def __iter__(self):
        curnode=self.head.next
        count=self.size
        while count:
            yield curnode.item
            curnode=curnode.next
            count-=1
        raise StopIteration
        
    def __str__(self):
        return str([i for i in iter(self)])
    
a=DoubleLinkedBase()
a.insert_first(1)
print(a)
a.delete_first()
print(a)

[1]
[]


可以看到基于双向链表能非常轻松的实现一个LIFO的栈,哨兵简化了链表的操作